## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ponta Do Sol,PT,32.6667,-17.1000,74.01,81,19,6.17,few clouds
1,1,Charlestown,AU,-32.9500,151.6667,46.69,78,61,5.99,broken clouds
2,2,Disraeli,CA,45.9001,-71.3491,77.22,39,18,1.01,few clouds
3,3,Qaanaaq,GL,77.4840,-69.3632,50.47,63,79,8.79,broken clouds
4,4,Tuatapere,NZ,-46.1333,167.6833,34.93,90,44,5.77,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ponta Do Sol,PT,32.6667,-17.1000,74.01,81,19,6.17,few clouds
10,10,Vila Velha,BR,-20.3297,-40.2925,72.55,76,11,11.48,few clouds
14,14,Savalou,BJ,7.9281,1.9756,74.26,91,89,7.78,overcast clouds
33,33,Lithakia,GR,37.7167,20.8333,74.57,73,0,2.21,clear sky
36,36,Souillac,MU,-20.5167,57.5167,70.27,82,88,15.19,light rain
42,42,Isangel,VU,-19.5500,169.2667,72.14,71,100,18.14,overcast clouds
46,46,Poum,NC,-20.2333,164.0167,71.76,58,0,24.56,clear sky
51,51,Srandakan,ID,-7.9386,110.2506,74.39,92,23,3.89,few clouds
52,52,Hasaki,JP,35.7333,140.8333,73.80,96,98,7.09,moderate rain
63,63,Rawlins,US,41.7911,-107.2387,72.43,44,100,18.90,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                85
City                   85
Country                82
Lat                    85
Lng                    85
Max Temp               85
Humidity               85
Cloudiness             85
Wind Speed             85
Current Description    85
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                82
City                   82
Country                82
Lat                    82
Lng                    82
Max Temp               82
Humidity               82
Cloudiness             82
Wind Speed             82
Current Description    82
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ponta Do Sol,PT,74.01,few clouds,32.6667,-17.1000,
10,Vila Velha,BR,72.55,few clouds,-20.3297,-40.2925,
14,Savalou,BJ,74.26,overcast clouds,7.9281,1.9756,
33,Lithakia,GR,74.57,clear sky,37.7167,20.8333,
36,Souillac,MU,70.27,light rain,-20.5167,57.5167,
42,Isangel,VU,72.14,overcast clouds,-19.5500,169.2667,
46,Poum,NC,71.76,clear sky,-20.2333,164.0167,
51,Srandakan,ID,74.39,few clouds,-7.9386,110.2506,
52,Hasaki,JP,73.80,moderate rain,35.7333,140.8333,
63,Rawlins,US,72.43,overcast clouds,41.7911,-107.2387,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
print('scanning...')
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
print('done')

scanning...
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
done


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ponta Do Sol,PT,74.01,few clouds,32.6667,-17.1000,Hotel do Campo
10,Vila Velha,BR,72.55,few clouds,-20.3297,-40.2925,Hotel Vitória Palace
14,Savalou,BJ,74.26,overcast clouds,7.9281,1.9756,Hotel F&F
33,Lithakia,GR,74.57,clear sky,37.7167,20.8333,Zante Plaza Hotel & Apartments
36,Souillac,MU,70.27,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
...,...,...,...,...,...,...,...
678,Yafran,LY,73.60,clear sky,32.0633,12.5286,بيوت الشباب بيفرن
679,Waingapu,ID,72.99,scattered clouds,-9.6567,120.2641,Pondok Wisata Elim
682,Karaton,ID,72.61,light rain,-6.3424,106.1045,Parkir Mobil Wisata CAS Water Park
685,Omsukchan,RU,71.85,clear sky,62.5333,155.8000,


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure

locations = clean_hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)


fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))